# SIR Model for Germany

Name: Aarya Sheakhar Yadav

Matrikelnummer : 419347

In [1]:
import pandas as pd
import numpy as np

import dash
dash.__version__
from dash import dcc
from dash import html
from dash.dependencies import Input, Output,State
import plotly.graph_objects as go
from scipy import optimize
from scipy import integrate

import os

from ipynb.fs.full.SIR_helpers import SIR_modelling, SIR_model_t, fit_odeint

In [2]:
df_analyse = pd.read_csv('../data/processed/COVID_final_set.csv', sep = ';')

In [3]:

fig = go.Figure()
app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''

    # SIR model for Germany

    '''),


    dcc.Dropdown(
        id = 'country_drop_down',
        options=[ {'label': each,'value':each} for each in df_analyse['country'].unique()],
        value= 'Germany', # which are pre-selected
        multi=False),

    dcc.Graph(figure = fig, id = 'SIR_graph')
    ])

In [4]:

def SIR(countries):

    SIR_modelling()


In [5]:

@app.callback(
    Output('SIR_graph', 'figure'),
    [Input('country_drop_down', 'value')])

def update_SIR_figure(country_drop_down):

    traces = []

    df_plot = df_analyse[df_analyse['country'] == country_drop_down]
    df_plot = df_plot[['state', 'country', 'confirmed', 'date']].groupby(['country', 'date']).agg(np.sum).reset_index()
    df_plot.sort_values('date', ascending = True).head()
    df_plot = df_plot.confirmed[35:]

    t, fitted = SIR_modelling(df_plot)

    traces.append(dict (x = t,
                        y = fitted,
                        mode = 'markers',
                        opacity = 0.9,
                        name = 'SIR-fit')
                  )

    traces.append(dict (x = t,
                        y = df_plot,
                        mode = 'lines',
                        opacity = 0.9,
                        name = 'Real data')
                  )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                title = 'SIR model fitting for COVID-19 data',

                xaxis= {'title':'Days',
                       'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis={'title': "Infected population"}
        )
    }


In [6]:


if __name__ == '__main__':
    app.run_server(debug = True, use_reloader = False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\hp\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning:

Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.

E:\DOCUMENTS\EDS\notebooks\SIR_helpers.ipynb:63: RuntimeWarning:

overflow encountered in double_scalars

E:\DOCUMENTS\EDS\notebooks\SIR_helpers.ipynb:64: RuntimeWarning:

overflow encountered in double_scalars

E:\DOCUMENTS\EDS\notebooks\SIR_helpers.ipynb:65: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\hp\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning:

Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.



standard deviation errors :  [1.18470243e-04 6.03166363e-05]  start infect: 21.0
Optimal parameters: beta = 0.015463367947782948  and gamma =  -0.004861794338783154
standard deviation errors :  [1.18470243e-04 6.03166363e-05]  start infect: 21.0
Optimal parameters: beta = 0.015463367947782948  and gamma =  -0.004861794338783154
standard deviation errors :  [73.04234047 72.90576332]  start infect: 1.0
Optimal parameters: beta = 1.005273466246028  and gamma =  0.9938970488729318
